In [ ]:
pip install openpyxl

In [ ]:
pip install xgboost

In [ ]:
import numpy as np
import pandas as pd
import openpyxl
import seaborn as sns
import matplotlib.pyplot as plt
import torch

from scipy.signal import periodogram

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import *
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

from xgboost import XGBRegressor

In [ ]:
data = pd.read_excel('./content/Train.xlsx', sheet_name='Monthly')

In [ ]:
data = data.rename(columns={'Unnamed: 0' : 'time'})  # переименовываем Unnamed: 0 в time (колонка время)
data.drop(0, inplace = True)  # удаляем неиформативную (и мешающую) строку преобразования данных
data['time'] = data['time'].apply(lambda x: x[:4] + x[5:]) # преобразовываем дату в формат datetime
data['time'] = pd.to_datetime(data['time'], format='%Y%m')
data = data.set_index(data['time'])
data = data.drop('time', axis=1)

scaler = StandardScaler()
scaler.fit(data)
data = pd.DataFrame(scaler.transform(data), index=data.index, columns=data.columns)

In [ ]:
def data_preparation(data, test=False):
    X = data.copy()
    X = X.rename(columns={'Unnamed: 0' : 'time'})  # переименовываем Unnamed: 0 в time (колонка время)
    datatime_function = lambda x: '20'+x[:2]+x[3:] if test else lambda x: x[:4] + x[5:]
    
    X['time'] = X['time'].apply(datatime_function) # преобразовываем дату в формат datetime
    X['time'] = pd.to_datetime(X['time'], format='%Y%m')
    X = X.set_index(X['time'])
    X = X.drop('time', axis=1)
    
    if not test:
        scaler = StandardScaler()
        scaler.fit(X)
        X = pd.DataFrame(scaler.transform(X), index=X.index, columns=X.columns)   
    return X

In [ ]:
data = data.astype(np.float64)

In [ ]:
np.sum(data.isna()).sort_values(ascending=False)  # смотрим какие колонки содержат пропущенные значения

In [ ]:
from sklearn import neighbors
from sklearn.linear_model import LinearRegression
# Заполним пропущенные значения регрессией

first_null = ["Обеспеченность оборота розничной торговли запасами, дней", "Товарные запасы в организациях розничной торговли, млрд.руб.", "Ввод в действие жилых домов, млн кв.м", "Реальные товарные запасы в организациях розничной торговли"]
first_null_date = first_null + ['date']
not_null_df = data[[item for item in data.columns if item not in first_null_date]]
plt.subplots(figsize=(10,10))

for null_val in first_null:
#     regress = neighbors.KNeighborsRegressor()
    regress = LinearRegression()
    regress.fit(not_null_df[24:], data[null_val][24:])
    predicted_values = regress.predict(not_null_df[:24])
    data.loc[:24, null_val] = predicted_values
    plt.plot(data[null_val], label=null_val)
    
plt.axvline(pd.Timestamp('2005-01-01'),color='r')
plt.legend()

In [ ]:
np.sum(np.sum(data.isna())) # пропущенных значений нет

#### Удалим коррелирующие признаки

In [ ]:
sns.heatmap(data.corr())

In [ ]:
drop_columns = []
TH = 0.95

for col1 in data.columns:
    for col2 in data.columns:
        if col1 != col2 and data[col1].corr(data[col2]) > TH:
            drop_columns.append(col2)

len(set(drop_columns))

In [ ]:
data1 = data.drop(set(drop_columns), axis=1)
data1.shape

#### Читаем тестовые данные (будем работать позже с этим)

In [ ]:
from sklearn.model_selection import train_test_split


def df_split_test_predict(df, forecast = 'Forecast'):
    return df[df.ne(forecast).all(1)], df[df.eq(forecast).any(1)]
def df_split_train_test(df, size = 0.2, forecast = 'Forecast'):
    return train_test_split(df[df.ne(forecast).all(1)], test_size=0.2, shuffle=False)

#### Модель корректировки разложения Фурье

In [ ]:
class model(torch.nn.Module):
    def __init__(self, periods):
        
        super(model, self).__init__()
        
        order = len(periods)
        
        self.periods = torch.tensor(periods)
        
        self.periods_sin = torch.nn.Parameter(torch.tensor(periods), requires_grad=True)
        self.periods_cos = torch.nn.Parameter(torch.tensor(periods), requires_grad=True)
        self.interceptions_sin = torch.nn.Parameter(torch.rand(order, requires_grad=True))
        self.interceptions_cos = torch.nn.Parameter(torch.rand(order, requires_grad=True))
        self.coefs_sin = torch.nn.Parameter(torch.rand(order, requires_grad=True))
        self.coefs_cos = torch.nn.Parameter(torch.rand(order, requires_grad=True))
        self.const = torch.nn.Parameter(torch.tensor(1., requires_grad=True))
        self.const_interception = torch.nn.Parameter(torch.tensor(1., requires_grad=True))
        
    def forward(self, X):
        y = 0
        for i in range(len(self.periods)):
            y += self.coefs_sin[i]*torch.sin(self.periods_sin[i]*X+self.interceptions_sin[i])
            y += self.coefs_cos[i]*torch.cos(self.periods_cos[i]*X+self.interceptions_cos[i])
        y += self.const*X
        y += self.const_interception
        return y

#### Функция возвращает функцию фичи

In [ ]:
def pearson_mse(x, y):  # смешанный лосс
    alpha = 5
    loss_mse = torch.nn.MSELoss()(x, y)
    vx = x - torch.mean(x)
    vy = y - torch.mean(y)
    pearson_loss = torch.sum(vx * vy) / (torch.sqrt(torch.sum(vx ** 2)) * torch.sqrt(torch.sum(vy ** 2)))
    return loss_mse + 1/alpha * (-pearson_loss)

In [ ]:
def feature_function(feature, print_loss=False):
    #########################
    #CONFIG 
    
    TH = 0.05  # какие периоды отсекаем по сравнению с максимумом
    max_periods = 3  # ограничение на максимальное кол-во периодов
    
    epochs = 10000
    lr=1e-2
    loss_function = pearson_mse
    #########################
    
    freqencies, spectrum = periodogram(  # разложение Фурье
        feature,
        fs=156,
        detrend='linear',
        window="boxcar",
        scaling='spectrum',
    )

    max_a = max(spectrum)
    n_periods = min(max_periods, np.sum(spectrum > max_a*TH))
    a = freqencies[np.argsort(spectrum)[:-n_periods-1:-1]]
    net = model(a)  # тренируем сеть корректировки
    y = torch.tensor(feature)
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    for epoch in range(epochs):
        out = net(torch.tensor(np.arange(0, 156)))
        loss = loss_function(out, y)
        opt.zero_grad()
        loss.backward()
        opt.step()
    if print_loss:
        print(loss)
    return net

## Посчитаем функции для фич

In [ ]:
import time
start = time.time()
funcs = []
for i in range(data1.shape[1]):
    feature = data1.iloc[:,i].values
    funcs.append(feature_function(feature, print_loss=False))
    print(i)
print(time.time()-start)

#### Теперь напишем функцию получения матрицы X по массиву дат

In [ ]:
def calc_all_time_X():
    return [[j] + [funcs[i](j-156) for i in range(len(funcs))] for j in range(1000)]

all_time_data = pd.DataFrame(calc_all_time_X(), columns=['month'] + list(data1.columns))

def calc_X(dates):
    base = pd.to_datetime(pd.Series(['01-01-1990' for i in range(dates.shape[0])]))
    base = base.dt.to_period('M').view(dtype='int64')
    
    num_dates = pd.Series(dates).dt.to_period('M').view(dtype='int64')-base
    data_result = all_time_data.iloc[num_dates[0]:num_dates[0]+dates.shape[0]]
    return pd.DataFrame(data_result)

#### Снижение размерности

In [ ]:
from sklearn.decomposition import PCA

X = calc_X(data1.index)
pca = PCA(n_components=10)
pca.fit(X)

In [ ]:
def test(model):
    metrics = {
        'mse' : mean_squared_error
    }
    losses = {}
    for key in metrics.keys():
        losses[key] = []
    
    for i in range(20, 30):
        try:
            test_data = pd.read_excel(f'./test_input/Test_input_{i}.xlsx', sheet_name='Monthly')
        except:
            continue
        test_data = data_preparation(test_data, test=True)
        train, test = df_split_train_test(test_data)
        X_train, X_test = pca.transform(calc_X(train.index)), pca.transform(calc_X(test.index))
        for var in range(1, test_data.shape[1]+1):
            
            train_var, test_var = train[f'Var{var}'], test[f'Var{var}']

            model.fit(X_train, train_var)
            pred = model.predict(X_test)

            for metric in metrics.keys():
                losses[metric].append(metrics[metric](test_var, pred))
            #print(i, var, mean_squared_error(test_var, pred))
    for metric in metrics.keys():
        losses[metric] = sum(losses[metric])/len(losses[metric])
            
    return losses

In [ ]:
def predict_test(path, model = LinearRegression(), excel_sheet = 'Monthly', forecast = 'Forecast'):
        try:
            test_data = pd.read_excel(path, sheet_name=excel_sheet)
        except:
#             print('Unvalid file. Check sheet name or file format')
            return pd.DataFrame()
        indexes = test_data['Unnamed: 0']
        test_data = data_preparation(test_data, test=True)
        
        for column in test_data.columns:
            var = test_data[column]
            train = var[var != forecast]
            pred = var[var == forecast]
            if pred.empty:
                continue
            if train.empty:
                print('Unvalid train data!!!')
                break
            X_train, X_pred = calc_X(train.index), calc_X(pred.index)

            model.fit(X_train, train)
            pred = model.predict(X_pred)
            test_data.loc[var[var == forecast].index, column] = pred
#         print(indexes)
#         print(test_data.index)
        test_data.index = indexes
        test_data.index.names = ['']
        return test_data

In [ ]:
from pandas import ExcelWriter

for i in range(0,4445):
    with ExcelWriter(f'./test_output/Test_output_{i}.xlsx') as writer:
        month = predict_test(f'./test_input/Test_input_{i}.xlsx', Lasso(alpha = 12), 'Monthly')
        if not month.empty:
            month.to_excel(writer, sheet_name='Monthly')
        else:
            print(f'No month in {i}')
        Quartel = predict_test(f'./test_input/Test_input_{i}.xlsx', Lasso(alpha = 12), 'Quarterly')
        if not Quartel.empty:
            print(f'Quarterly in {i}')
            Quartel.to_excel(writer, sheet_name='Quarterly')
# 4445